교체하는 작업은 dict를 사용해서 key:value 형태로 교체하면 그만일 듯

In [1]:
import numpy as np
import pandas as pd

import os
import datetime

In [2]:
money_data = pd.read_csv("./data/2021_money_add.csv", encoding="EUC-KR", header=0)
train_data = pd.read_csv("./data/2021_subway_transport_data.csv", encoding="EUC-KR", header=0)
money_data.head()

,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,자치구,동,상권_코드_명,서비스_업종_코드,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,0,2021,1,A,골목상권,2110001,종로구,평창동,이북5도청사,CS100001,...,157,15667,7253,108,1098,2013,4344,7792,7568,28
1,1,2021,1,A,골목상권,2110001,종로구,평창동,이북5도청사,CS100008,...,0,6679,4293,52,926,1222,2698,3404,2670,9
2,2,2021,1,A,골목상권,2110001,종로구,평창동,이북5도청사,CS100009,...,9,124,36,0,0,9,18,62,71,3
3,3,2021,1,A,골목상권,2110001,종로구,평창동,이북5도청사,CS100010,...,8,356,881,0,243,260,213,288,235,5
4,4,2021,1,A,골목상권,2110001,종로구,평창동,이북5도청사,CS200001,...,0,56,72,0,0,8,96,8,16,3


In [3]:
train_data.head()

,연번,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시 이후,합 계
0,1,2021-01-01,1,150,서울역,승차,86,111,157,306,...,837,796,839,658,579,479,510,445,101,9339
1,2,2021-01-01,1,150,서울역,하차,85,355,438,592,...,781,710,626,437,425,354,307,134,77,9951
2,3,2021-01-01,1,151,시청,승차,43,45,75,111,...,227,233,347,294,186,216,204,110,29,2947
3,4,2021-01-01,1,151,시청,하차,40,145,176,312,...,158,140,152,121,57,54,64,23,17,2700
4,5,2021-01-01,1,152,종각,승차,49,59,89,126,...,513,548,504,572,492,593,513,130,31,5686


In [4]:
print(len(money_data[money_data['상권_코드_명'].str.contains("역")]))
print(len(money_data[money_data['상권_코드_명'].str.contains(r".역")]))

54448
54232


In [5]:
# money_data[money_data['상권_코드_명'].str.contains(r".역")]['상권_코드_명'].unique()  # 정규표현식 사용해서 ~~역으로 끝나는 것만 추출

In [42]:
station_money = pd.DataFrame(money_data[money_data['상권_코드_명'].str.contains("역")]['상권_코드_명'].unique())
# bakery_data.index[(bakery_data['상권_코드_명'].str.contains("DMC"))]
station_money_add = pd.DataFrame(money_data[(money_data['상권_코드_명'].str.contains("DMC"))]['상권_코드_명'].unique())
station_money_add2 = pd.DataFrame(money_data[(money_data['상권_코드_명'].str.contains("버티고개"))]['상권_코드_명'].unique())
station_money = pd.concat([station_money, station_money_add], axis=0, ignore_index=True)
station_money = pd.concat([station_money, station_money_add2], axis=0, ignore_index=True)
station_money.columns = ["상권코드명",]
# print(station_money.iloc[1], type(station_money.iloc[0]))
# print(station_money.iloc[1].astype('str').str.split('역'))

# tmp = station_money.iloc[1].astype('str').str.split('역')
# print(len(tmp[0]))
# print(tmp.str.get(0))

# print(len(station_money))

# for i in range(len(station_money)):
#     splited = station_money.iloc[i].astype('str').str.split("역")
#     for j in range(len(splited[0])):
#         col_name = str(j+1) + "번 분리 데이터" 
#         if col_name in station_money.columns:
#             continue
#         else:
#             station_money[col_name] = None
#     for j in range(len(splited[0])):
#         col_name = str(j+1) + "번 분리 데이터" 
#         station_money[col_name].iloc[i] = splited.str.get(j)

col_arr = []

splt = station_money["상권코드명"].str.split(r"역$|역 |역\(|역,|역_|역골|역지|역전|역1|\(| |역상|역둘|역동|역\)|역교|C\(|\)", expand=True)
for i in range(len(splt.columns)):
    col_name = str(i+1) + "번 분리 데이터" 
    col_arr.append(col_name)
splt.columns = col_arr

origin_station_money = station_money.copy()
station_money = pd.concat([station_money, splt], axis=1)

# station_money.index[(station_money['상권코드명'].str.contains("DMC"))]
station_money
station_money.to_csv("./station_data_moneydata.csv", encoding="utf-8-sig")

In [7]:
station_train = pd.DataFrame(train_data['역명'].unique())
station_train.columns = ["역명",]
# station_train

col_arr_train = []

splt_train = station_train["역명"].str.split(r"\(|역$", expand=True)
for i in range(len(splt_train.columns)):
    col_name = str(i+1) + "번 분리 데이터"
    col_arr_train.append(col_name)
splt_train.columns = col_arr_train

origin_station_train = station_train.copy()
station_train = pd.concat([station_train, splt_train], axis=1)

station_train.to_csv("./station_data_traindata.csv", encoding="utf-8-sig")

In [8]:
# station_money['1번 분리 데이터'].unique()
# '서울' in station_money['1번 분리 데이터'].unique()

In [9]:
# station_train['1번 분리 데이터'].unique()
# '서울' in station_train['1번 분리 데이터'].unique()

In [10]:
# stm_arr = station_money['1번 분리 데이터'].unique()
# stt_arr = station_train['1번 분리 데이터'].unique()

# stm_arr_origin = stm_arr.copy()
# stt_arr_origin = stt_arr.copy()

# # print(type(stm_arr))
# # print(len(stm_arr))

# matching_idx_arr = []

# for i in range(len(stt_arr)):
#     # print(len(stt_arr))
#     for j in range(len(stm_arr)):
#         # print(len(stm_arr))
#         if stt_arr[i] == stm_arr[j]:
#             matching_idx_arr.append(j)
#             stm_arr = np.delete(stm_arr, j)
#             break
# print(matching_idx_arr)
# stm_arr

# # 이런 느낌으로.. 분리한 데이터를 

In [43]:
# 예상 구동시간 20초
stt_arr = station_train['1번 분리 데이터'].unique()

# pop 사용, 실제 pop는 column이니까, concat으로
poped_df = pd.DataFrame()

for i in range(len(stt_arr)):
    if stt_arr[i] == '둔촌동':
        stt_arr[i] = '둔촌' or '둔촌동'
    for j in range(len(station_money.columns)):
        for k in range(len(station_money)):
            if station_money.iloc[k, j] == stt_arr[i]:
                poped = pd.DataFrame(station_money.iloc[k]).T
                poped['idx'] = j
                poped_df = pd.concat([poped_df, poped], axis=0, ignore_index=True)
                station_money.drop([station_money.index[k]], inplace=True, axis=0)
                break
            
station_money
# 341 312 122 120 120 118 118 118

,상권코드명,1번 분리 데이터,2번 분리 데이터,3번 분리 데이터,4번 분리 데이터,5번 분리 데이터
4,동대문역 3번,동대문,3번,None,None,None
6,창신역 4번,창신,4번,None,None,None
8,동묘앞역 3번,동묘앞,3번,None,None,None
14,청구역 3번,청구,3번,None,None,None
17,효창공원앞역 5번,효창공원앞,5번,None,None,None
...,...,...,...,...,...,...
537,선유도역 골목형상점가,선유도,골목형상점가,None,None,None
538,영등포역지하도상점가,영등포,하도상점가,None,None,None
539,남성역골목시장,남성,목시장,None,None,None
540,강남터미널지하도상점가2구역,강남터미널지하도상점가2구,,None,None,None


In [44]:
# 예상 구동시간 60초
prelen = 0
cnt = 0
loops = 0

while len(station_money) != prelen or cnt < 3:
    if loops != 0:
        prelen = len(station_money)
    # print("이전길이", prelen)

    for i in range(len(stt_arr)):
        if stt_arr[i] == '둔촌동':
            stt_arr[i] = '둔촌' or '둔촌동'
        for j in range(len(station_money.columns)):
            for k in range(len(station_money)):
                if station_money.iloc[k, j] == stt_arr[i]:
                    poped = pd.DataFrame(station_money.iloc[k]).T
                    poped['idx'] = j
                    poped_df = pd.concat([poped_df, poped], axis=0, ignore_index=True)
                    station_money.drop([station_money.index[k]], inplace=True, axis=0)
                    break
    
    if prelen == len(station_money):
        cnt += 1
    else:
        cnt = 0

    loops += 1
    # print("현재길이",len(station_money))
    # print("반복횟수", cnt)

# 남은것 보기
station_money
# 217 151 127 123 121 121 112

,상권코드명,1번 분리 데이터,2번 분리 데이터,3번 분리 데이터,4번 분리 데이터,5번 분리 데이터
20,남영역 1번,남영,1번,None,None,None
25,서빙고역 1번,서빙고,1번,None,None,None
38,서울숲역 1번,서울숲,1번,None,None,None
57,회기역 1번,회기,1번,None,None,None
58,외대앞역 1번,외대앞,1번,None,None,None
...,...,...,...,...,...,...
511,송파나루역,송파나루,,None,None,None
536,역촌중앙시장,역촌중앙시장,None,None,None,None
537,선유도역 골목형상점가,선유도,골목형상점가,None,None,None
538,영등포역지하도상점가,영등포,하도상점가,None,None,None


In [13]:
# poped_df

In [14]:
station_money.to_csv("./station_data_testing.csv", encoding='utf-8-sig')
# poped_df.to_csv("./poped_data.csv", encoding='utf-8-sig')

In [15]:
stt_dict = dict(zip(stt_arr, range(len(stt_arr))))

# # stt_dict
# stt_dict['남위례']

In [45]:
poped_df_origin = poped_df.copy()
poped_df['link'] = None

for i in range(len(poped_df)):
    col_idx = poped_df.iloc[i]['idx']
    linked_data = poped_df.iloc[i, col_idx]
    
    poped_df.iloc[i, -1] = stt_dict[linked_data]

poped_df

,상권코드명,1번 분리 데이터,2번 분리 데이터,3번 분리 데이터,4번 분리 데이터,5번 분리 데이터,idx,link
0,서울역 15번,서울,15번,None,None,None,1,0
1,시청역_1번,시청,1번,None,None,None,1,1
2,북창동(시청역_6번),북창동,시청,6번,,None,2,1
3,종각역,종각,,None,None,None,1,2
4,종로3가역,종로3가,,None,None,None,1,3
...,...,...,...,...,...,...,...,...
427,굽은다리역,굽은다리,,None,None,None,1,144
428,내방역 1번,내방,1번,None,None,None,1,206
429,암사역,암사,,None,None,None,1,230
430,연신내역,연신내,,None,None,None,1,60


In [46]:
poped_df.to_csv("./poped_data.csv", encoding='utf-8-sig')

In [47]:
linked_df = poped_df.loc[:,['상권코드명', 'link']]

linked_df

,상권코드명,link
0,서울역 15번,0
1,시청역_1번,1
2,북창동(시청역_6번),1
3,종각역,2
4,종로3가역,3
...,...,...
427,굽은다리역,144
428,내방역 1번,206
429,암사역,230
430,연신내역,60


In [48]:
linked_df.to_csv("./sales_district_name_IDcode.csv", encoding='utf-8-sig')

In [20]:
stt_df = station_train

if 'link' in stt_df.columns:
    del stt_df['link']

stt_df['link'] = None
for i in range(len(stt_df)):
    linked_data = stt_df.iloc[i]['1번 분리 데이터']
    
    if linked_data == '둔촌동':
        linked_data = '둔촌' or '둔촌동'
        
    stt_df.iloc[i, -1] = stt_dict[linked_data]
    
stt_df

,역명,1번 분리 데이터,2번 분리 데이터,link
0,서울역,서울,,0
1,시청,시청,None,1
2,종각,종각,None,2
3,종로3가,종로3가,None,3
4,종로5가,종로5가,None,4
...,...,...,...,...
244,신내,신내,None,244
245,강일,강일,None,245
246,하남검단산,하남검단산,None,246
247,하남시청(덕풍·신장),하남시청,덕풍·신장),247


기준_년_코드	기준_분기_코드	상권_구분_코드	상권_구분_코드_명	상권_코드	상권_코드_명	서비스_업종_코드	서비스_업종_코드_명	분기당_매출_금액	분기당_매출_건수	...	시간대_건수~24_매출_건수	남성_매출_건수	여성_매출_건수	연령대_10_매출_건수	연령대_20_매출_건수	연령대_30_매출_건수	연령대_40_매출_건수	연령대_50_매출_건수	연령대_60_이상_매출_건수	점포수

In [21]:
stt_df.to_csv("./subway_station_name_IDcode.csv", encoding='utf-8-sig')

In [22]:
linked_stt_df = stt_df[['역명', 'link']]
linked_stt_df.to_csv("./subway_station_name_IDcode.csv", encoding='utf-8-sig')

In [23]:
# testing_data = money_data[money_data['상권_코드_명'].str.contains("역")].iloc[0:100]
testing_data = money_data[money_data['상권_코드_명'].str.contains("역")].iloc[0:10]

if 'link' in testing_data.columns:
    del testing_data['link']

testing_data['link'] = None

# 추가
testing_data.reset_index(drop=True, inplace=True)

# 추가 종료

for i in range(len(testing_data)):
    for j in range(len(linked_df)):
        # if testing_data.iloc[i, 5] == linked_df.iloc[j, 0]:
        if testing_data.loc[i, '상권_코드_명'] == linked_df.iloc[j, 0]:
            # testing_data.iloc[i, -1] = linked_df.iloc[j, 1]
            testing_data.loc[i, 'link'] = linked_df.iloc[j, 1]

result_df = testing_data[['상권_코드_명', '서비스_업종_코드_명', '점포수', 'link']]

result_df
# 상권_코드_명

,상권_코드_명,서비스_업종_코드_명,점포수,link
0,독립문역 1번,한식음식점,29,65
1,독립문역 1번,제과점,9,65
2,독립문역 1번,치킨전문점,9,65
3,독립문역 1번,커피-음료,9,65
4,독립문역 1번,일반교습학원,12,65
5,독립문역 1번,외국어학원,9,65
6,독립문역 1번,치과의원,3,65
7,독립문역 1번,골프연습장,1,65
8,독립문역 1번,PC방,3,65
9,독립문역 1번,자동차수리,4,65


,날짜,호선,역번호,역명,구분,
06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,10시-11시,
11시-12시,12시-13시,13시-14시,14시-15시,15시-16시,16시-17시,
17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시 이후,
합 계,0시11시,11시17시,17시24시,06시11시,연,월,일,요일

In [24]:
# # 예상시간 6분 30초
# arrive_data = pd.read_csv("./data/2021_arrive_qt_tz_01.csv", encoding='utf-8')
# linked_stt_df = stt_df[['역명', 'link']]
# # linked_stt_df

# arrive_data_origin = arrive_data.copy()

# if 'link' in arrive_data.columns:
#     del arrive_data['link']

# arrive_data['link'] = None

# for i in range(len(arrive_data)):
#     for j in range(len(linked_stt_df)):
#         # if arrive_data.iloc[i, 4] == linked_stt_df.iloc[j, 0]:
#         if arrive_data.loc[i, '역명'] == linked_stt_df.iloc[j, 0]:
#             # arrive_data.iloc[i, -1] = linked_stt_df.iloc[j, 1]
#             arrive_data.loc[i, 'link'] = linked_stt_df.iloc[j, 1]

# result_stt_df = arrive_data[['역명','00시11시','11시17시','17시24시','합 계','link']]

# result_stt_df

,역명,00시11시,11시17시,17시24시,합 계,link
0,서울역,3058,4533,2360,9951,0
1,시청,1085,1127,488,2700,1
2,종각,1385,2601,1001,4987,2
3,종로3가,1050,3127,889,5066,3
4,종로5가,756,2342,639,3737,4
...,...,...,...,...,...,...
24999,남한산성입구(성남법원.검찰청),2692,3190,7136,13018,239
25000,단대오거리,2351,2525,5612,10488,240
25001,신흥,932,1276,2520,4728,241
25002,수진,979,1306,2466,4751,242


In [25]:
# merge_test_df = testing_data[['상권_코드_명', '서비스_업종_코드_명', '점포수', '분기당_매출_건수', 'link']]
# merge_test_stt_df = arrive_data[['역명','00시11시','11시17시','17시24시','합 계','link']]

In [26]:
# concat 잘못되어서 주석처리했음.
# final_result = pd.DataFrame()

# for i in range(len(merge_test_df)):
#     money_target = merge_test_df.iloc[i].T
#     for j in range(len(merge_test_stt_df)):
#         people_target = merge_test_stt_df.iloc[j].T
#         if money_target['link'] == people_target['link']:
#             final_result = pd.concat([final_result,money_target,people_target],axis=1, ignore_index=True)

# final_result

,0,1,2,3,4,5,6,7,8,9,...,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799
상권_코드_명,독립문역 1번,NaN,독립문역 1번,NaN,독립문역 1번,NaN,독립문역 1번,NaN,독립문역 1번,NaN,...,독립문역 1번,NaN,독립문역 1번,NaN,독립문역 1번,NaN,독립문역 1번,NaN,독립문역 1번,NaN
서비스_업종_코드_명,한식음식점,NaN,한식음식점,NaN,한식음식점,NaN,한식음식점,NaN,한식음식점,NaN,...,자동차수리,NaN,자동차수리,NaN,자동차수리,NaN,자동차수리,NaN,자동차수리,NaN
점포수,29,NaN,29,NaN,29,NaN,29,NaN,29,NaN,...,4,NaN,4,NaN,4,NaN,4,NaN,4,NaN
분기당_매출_건수,16676,NaN,16676,NaN,16676,NaN,16676,NaN,16676,NaN,...,1173,NaN,1173,NaN,1173,NaN,1173,NaN,1173,NaN
link,65,65,65,65,65,65,65,65,65,65,...,65,65,65,65,65,65,65,65,65,65
역명,NaN,독립문,NaN,독립문,NaN,독립문,NaN,독립문,NaN,독립문,...,NaN,독립문,NaN,독립문,NaN,독립문,NaN,독립문,NaN,독립문
00시11시,NaN,506,NaN,854,NaN,639,NaN,1843,NaN,1936,...,NaN,1697,NaN,1032,NaN,2558,NaN,2684,NaN,2766
11시17시,NaN,925,NaN,1509,NaN,1062,NaN,1694,NaN,1587,...,NaN,1964,NaN,1471,NaN,1911,NaN,2079,NaN,2208
17시24시,NaN,634,NaN,929,NaN,818,NaN,2192,NaN,2218,...,NaN,1656,NaN,1265,NaN,2696,NaN,2887,NaN,2807
합 계,NaN,2065,NaN,3292,NaN,2519,NaN,5729,NaN,5741,...,NaN,5317,NaN,3768,NaN,7165,NaN,7650,NaN,7781


In [27]:
merge_test_df = testing_data[['상권_코드_명', '서비스_업종_코드_명', '점포수', '분기당_매출_건수', 'link']]
# merge_test_df.columns = ['상권_코드_명', '서비스_업종_코드_명', '점포수', '분기당_매출_건수', 'linkID']
merge_test_df = pd.DataFrame(merge_test_df[merge_test_df['서비스_업종_코드_명'].str.contains("제과")])
# merge_test_df = pd.DataFrame(merge_test_df[merge_test_df['상권_코드_명'].str.contains("독립문")])
# station_money = pd.DataFrame(money_data[money_data['상권_코드_명'].str.contains("역")]['상권_코드_명'].unique())

merge_test_stt_df = arrive_data[['역명','00시11시','11시17시','17시24시','합 계','link']]
# merge_test_stt_df.columns = ['역명','00시11시','11시17시','17시24시','합 계','linkID']
# merge_test_stt_df.index[(merge_test_stt_df['link']==65) | (merge_test_stt_df['link']==174)]
# merge_test_stt_df['link'][[67,198]]
# print(merge_test_df)
merge_test_stt_df = merge_test_stt_df.iloc[[67,198],:]

# print (df.index[(df['B'] == 19) | (df['C'] == 19)].tolist())

tmp_df = pd.DataFrame()
final_result = pd.DataFrame()

tmp_df = pd.merge(merge_test_df, merge_test_stt_df, how='outer', on='link')
final_result = pd.concat([final_result, tmp_df],axis=1)
        # final_result = pd.concat([final_result, people_target],axis=1, ignore_index=True)

final_result

C:\Users\La_Elya\AppData\Local\Temp\ipykernel_19968\1439714585.py:19: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  tmp_df = pd.merge(merge_test_df, merge_test_stt_df, how='outer', on='link')


,상권_코드_명,서비스_업종_코드_명,점포수,분기당_매출_건수,link,역명,00시11시,11시17시,17시24시,합 계
0,독립문역 1번,제과점,9.0,50998.0,65.0,독립문,506,925,634,2065
1,NaN,NaN,NaN,NaN,174,창신,132,264,313,709


In [28]:
#예상시간 1분 10초
bakery_data = money_data[money_data['서비스_업종_코드_명'].str.contains("제과")]
bakery_data.reset_index(drop=True, inplace=True)
# bakery_data

if 'link' in bakery_data.columns:
    del bakery_data['link']

bakery_data['link'] = None

for i in range(len(bakery_data)):
    for j in range(len(linked_df)):
        if bakery_data.loc[i, '상권_코드_명'] == linked_df.iloc[j, 0]:
            bakery_data.loc[i, 'link'] = linked_df.iloc[j, 1]

C:\Users\La_Elya\AppData\Local\Temp\ipykernel_19968\2739844580.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bakery_data['link'] = None


In [29]:
bakery_data.head()
# len(bakery_data)

,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,자치구,동,상권_코드_명,서비스_업종_코드,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,link
0,13,2021,1,A,골목상권,2110002,종로구,무악동,독립문역 1번,CS100005,...,18699,30518,368,9544,11114,11214,10826,6153,9,65
1,30,2021,1,A,골목상권,2110003,종로구,부암동,세검정초등학교,CS100005,...,5502,7926,82,1746,2147,3410,3668,2378,3,None
2,57,2021,1,A,골목상권,2110005,종로구,부암동,세검정,CS100005,...,2169,4182,6,612,955,1713,1971,1095,3,None
3,76,2021,1,A,골목상권,2110006,종로구,부암동,부암동주민센터,CS100005,...,6424,12791,80,4458,5347,3985,3725,1620,3,None
4,88,2021,1,A,골목상권,2110008,종로구,청운효자동,배화여자대학교(박노수미술관),CS100005,...,11043,27380,335,14968,11005,5381,4832,1902,18,None


In [30]:
# bakery_data.index[(bakery_data['상권_코드_명'].str.contains("DMC"))]

In [31]:
# 분기별로 매출 나누기
# tsda = bakary_data.iloc[0:100]
bd_qt1 = bd_qt2 = bd_qt3 = bd_qt4 = pd.DataFrame()

for i in range(len(bakery_data)):
    target_row = pd.DataFrame(bakery_data.iloc[i]).T
    target_qt = bakery_data.loc[i, '기준_분기_코드']
    for j in range(4):
        if target_qt == j+1:
            globals()["bd_qt"+str(j+1)] = pd.concat([globals()["bd_qt"+str(j+1)], target_row], axis=0, ignore_index=True)


In [32]:
# arrive_data = pd.read_csv("./data/2021_arrive_qt_tz_01.csv", encoding='utf-8')
bd_qt1['link'].value_counts() # 172
dna_bd_qt1 = bd_qt1.dropna(axis=0, inplace=False)
dna_bd_qt1['link'].value_counts() # 172

63     4
150    4
118    4
34     4
1      3
      ..
99     1
3      1
67     1
2      1
133    1
Name: link, Length: 174, dtype: int64

In [33]:
group_test = dna_bd_qt1.groupby(['link']).agg('sum')
group_test = group_test.reset_index()
group_test

,link,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,자치구,동,상권_코드_명,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,0,21571,2021,1,D,발달상권,2120043,중구,회현동,서울역,...,3017,38408,55607,125,20137,36415,21500,11640,4196,27
1,1,62783,6063,3,DDD,발달상권발달상권발달상권,6360063,중구중구중구,소공동소공동소공동,시청역_1번시청역_8번북창동(시청역_6번),...,2886,33519,41222,362,12826,22516,20433,13098,5500,23
2,2,20452,2021,1,D,발달상권,2120006,종로구,종로1.2.3.4가동,종각역,...,4614,98518,116299,2233,64337,68232,45466,25070,9476,27
3,3,20565,2021,1,D,발달상권,2120009,종로구,종로1.2.3.4가동,종로3가역,...,5866,22957,27823,370,15427,11333,8998,8610,6042,20
4,4,20721,2021,1,D,발달상권,2120014,종로구,종로5.6가동,종로5가역,...,1026,6344,6756,126,1818,2543,2607,3078,2927,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,232,28576,2021,1,D,발달상권,2120233,송파구,방이2동,몽촌토성역,...,87,2219,3624,73,1160,1493,1936,887,294,3
170,233,65496,6063,3,AAD,골목상권골목상권발달상권,6342250,송파구송파구송파구,석촌동석촌동송파1동,석촌역 7번석촌역 6번석촌역(석촌호수),...,5894,20637,32400,417,11411,16084,10644,7522,6958,12
171,234,28490,2021,1,D,발달상권,2120230,송파구,석촌동,송파사거리(송파역),...,18481,52755,76197,3420,27872,37490,30580,20825,8765,18
172,235,28641,2021,1,D,발달상권,2120235,송파구,문정2동,문정역,...,1321,35719,29629,1964,18310,25564,10502,6527,2481,16


In [39]:
group_test_origin = group_test.copy()
# tmp_df = pd.merge(merge_test_df, merge_test_stt_df, how='outer', on='link')

group_test = dna_bd_qt1.groupby(['link']).agg('sum')
group_test = group_test.reset_index()

group_test = pd.merge(group_test, linked_stt_df, how='outer', on='link')
group_test.dropna(axis=0, inplace=True)
group_test.drop(columns=['상권_코드_명'], axis=1, inplace=True)

# group_test['link'] = group_test['link'].astype('int64')
# group_test['기준_년_코드'] = group_test['기준_년_코드'].astype('int64')
# group_test['상권_코드'] = group_test['상권_코드'].astype('int64')
# group_test['기준_분기_코드'] = group_test['기준_분기_코드'].astype('int64')
# group_test['점포수'] = group_test['점포수'].astype('int64')
# 한번에 하려면 dtype으로 float들 걸러내고, 그걸 다시 int로 바꾸는 방식을 사용하면 되려나?

target_float_arr = []
for i in group_test.columns:
    if group_test[i].dtype == (r"float"):
        target_float_arr.append(i)

# print(target_float_arr)

for i in target_float_arr:
    group_test[i] = group_test[i].astype('int64')

miss_sum_arr_num = ['기준_분기_코드','기준_년_코드','상권_코드']
miss_sum_arr_str = ['상권_구분_코드_명','서비스_업종_코드_명','서비스_업종_코드','상권_구분_코드','자치구','동']

for i in range(len(group_test)):
    total_sum = group_test.loc[i, '기준_분기_코드']
    for j in miss_sum_arr_num:
        group_test.loc[i, j] = int(group_test.loc[i, j] / total_sum)

    for j in miss_sum_arr_str:
        group_test.loc[i, j] = group_test.loc[i, j][0:(int(len(group_test.loc[i, j])/total_sum))]

    # group_test.loc[i, '기준_년_코드'] = int(group_test.loc[i, '기준_년_코드'] / total_sum)
    # group_test.loc[i, '상권_코드'] = int(group_test.loc[i, '상권_코드'] / total_sum)
    # group_test.loc[i, '상권_구분_코드_명'] = group_test.loc[i, '상권_구분_코드_명'][0:(int(len(group_test.loc[i, '상권_구분_코드_명'])/total_sum))]
    # group_test.loc[i, '서비스_업종_코드_명'] = group_test.loc[i, '서비스_업종_코드_명'][0:(int(len(group_test.loc[i, '서비스_업종_코드_명'])/total_sum))]
    # group_test.loc[i, '서비스_업종_코드'] = group_test.loc[i, '서비스_업종_코드'][0:(int(len(group_test.loc[i, '서비스_업종_코드'])/total_sum))]
    # group_test.loc[i, '상권_구분_코드'] = group_test.loc[i, '상권_구분_코드'][0:(int(len(group_test.loc[i, '상권_구분_코드'])/total_sum))]
    
group_test

C:\Users\La_Elya\AppData\Local\Temp\ipykernel_19968\3262542958.py:7: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  group_test = pd.merge(group_test, linked_stt_df, how='outer', on='link')


,link,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,자치구,동,서비스_업종_코드,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,역명
0,0.0,21571,2021,1,D,발달상권,2120043,중구,회현동,CS100005,...,38408,55607,125,20137,36415,21500,11640,4196,27,서울역
1,1.0,62783,2021,1,D,발달상권,2120021,중구,소공동,CS100005,...,33519,41222,362,12826,22516,20433,13098,5500,23,시청
2,2.0,20452,2021,1,D,발달상권,2120006,종로구,종로1.2.3.4가동,CS100005,...,98518,116299,2233,64337,68232,45466,25070,9476,27,종각
3,3.0,20565,2021,1,D,발달상권,2120009,종로구,종로1.2.3.4가동,CS100005,...,22957,27823,370,15427,11333,8998,8610,6042,20,종로3가
4,4.0,20721,2021,1,D,발달상권,2120014,종로구,종로5.6가동,CS100005,...,6344,6756,126,1818,2543,2607,3078,2927,6,종로5가
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,232.0,28576,2021,1,D,발달상권,2120233,송파구,방이2동,CS100005,...,2219,3624,73,1160,1493,1936,887,294,3,몽촌토성(평화의문)
170,233.0,65496,2021,1,A,골목상권,2114083,송파구,석촌동,CS100005,...,20637,32400,417,11411,16084,10644,7522,6958,12,석촌
171,234.0,28490,2021,1,D,발달상권,2120230,송파구,석촌동,CS100005,...,52755,76197,3420,27872,37490,30580,20825,8765,18,송파
172,235.0,28641,2021,1,D,발달상권,2120235,송파구,문정2동,CS100005,...,35719,29629,1964,18310,25564,10502,6527,2481,16,문정


In [40]:
# 예상 구동시간 22초
qt1_popular_data = pd.read_csv("./data/2021_1qt.csv", encoding='utf-8', header=0)

if 'link' in qt1_popular_data.columns:
    del qt1_popular_data['link']

qt1_popular_data['link'] = None

for i in range(len(qt1_popular_data)):
    for j in range(len(linked_stt_df)):
        if qt1_popular_data.loc[i, '역명'] == linked_stt_df.iloc[j, 0]:
            qt1_popular_data.loc[i, 'link'] = linked_stt_df.iloc[j, 1]

qt1_popular_data

,역명,요일,00시11시,11시17시,17시24시,총합,평균,link
0,가락시장,금,71509,58189,73629,203327,67776.0,85
1,가락시장,목,73742,55091,67826,196659,65553.0,85
2,가락시장,수,83134,63386,77969,224489,74830.0,85
3,가락시장,월,79734,58693,70279,208706,69569.0,85
4,가락시장,일,16379,37958,35688,90025,30008.0,85
...,...,...,...,...,...,...,...,...
1726,효창공원앞,수,25693,20390,33690,79773,26591.0,169
1727,효창공원앞,월,24387,18515,30888,73790,24597.0,169
1728,효창공원앞,일,4764,13171,13891,31826,10609.0,169
1729,효창공원앞,토,8195,17990,17825,44010,14670.0,169


In [36]:
# qt1_popular_data.iloc[0]
# wd_name = qt1_popular_data.loc[0, '요일']
testing2 = qt1_popular_data.copy()

# testing2 = testing2.iloc[:10]

# testing2['합친횟수'] = 1
timezone = ['00시11시','11시17시','17시24시','총합', '평균']
weekdays_name =['월','화','수','목','금','토','일']
col_names_arr = []

for i in weekdays_name:
    for j in timezone:
        new_col_name = i + "_" + j
        col_names_arr.append(new_col_name)

col_names_arr

for i in col_names_arr:
    testing2[i] = 0

for i in range(len(testing2)):
    wd_name = testing2.loc[i, '요일']
    # print()
    # print("df요일",wd_name)
    for j in col_names_arr:
        # print("타겟컬럼",j)
        # print("타겟컬럼 요일",j[0])
        if wd_name==j[0]:
            testing2.loc[i, j] = testing2.loc[i, j[2:]] 
        # for k in weekdays_name:
        #     # print("요일",k)
        #     if wd_name==k and wd_name==j[0]:
        #         # print("df요일",wd_name, "/","요일",k)
        #         testing2[j] =testing2[j[2:]]

# testing2[new_col_name] = testing2.loc[0, timezone[3]]

# testing2.index[(testing2['요일'] == "금")]
testing2


,역명,요일,00시11시,11시17시,17시24시,총합,평균,link,월_00시11시,월_11시17시,...,토_00시11시,토_11시17시,토_17시24시,토_총합,토_평균,일_00시11시,일_11시17시,일_17시24시,일_총합,일_평균
0,가락시장,금,71509,58189,73629,203327,67776.0,85,0,0,...,0,0,0,0,0,0,0,0,0,0
1,가락시장,목,73742,55091,67826,196659,65553.0,85,0,0,...,0,0,0,0,0,0,0,0,0,0
2,가락시장,수,83134,63386,77969,224489,74830.0,85,0,0,...,0,0,0,0,0,0,0,0,0,0
3,가락시장,월,79734,58693,70279,208706,69569.0,85,79734,58693,...,0,0,0,0,0,0,0,0,0,0
4,가락시장,일,16379,37958,35688,90025,30008.0,85,0,0,...,0,0,0,0,0,16379,37958,35688,90025,30008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,효창공원앞,수,25693,20390,33690,79773,26591.0,169,0,0,...,0,0,0,0,0,0,0,0,0,0
1727,효창공원앞,월,24387,18515,30888,73790,24597.0,169,24387,18515,...,0,0,0,0,0,0,0,0,0,0
1728,효창공원앞,일,4764,13171,13891,31826,10609.0,169,0,0,...,0,0,0,0,0,4764,13171,13891,31826,10609
1729,효창공원앞,토,8195,17990,17825,44010,14670.0,169,0,0,...,8195,17990,17825,44010,14670,0,0,0,0,0


In [37]:
testing3 = testing2.groupby(['역명', 'link']).agg('sum')
testing3.drop(['00시11시','11시17시','17시24시','총합', '평균'], axis=1, inplace=True)
testing3.reset_index(drop=False, inplace=True)
testing3

C:\Users\La_Elya\AppData\Local\Temp\ipykernel_19968\1795566976.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  testing3 = testing2.groupby(['역명', 'link']).agg('sum')


,역명,link,월_00시11시,월_11시17시,월_17시24시,월_총합,월_평균,화_00시11시,화_11시17시,화_17시24시,...,토_00시11시,토_11시17시,토_17시24시,토_총합,토_평균,일_00시11시,일_11시17시,일_17시24시,일_총합,일_평균
0,가락시장,85,79734,58693,70279,208706,69569,83778,62198,75902,...,30932,57325,39730,127987,42662,16379,37958,35688,90025,30008
1,가산디지털단지,216,444386,74018,51569,569973,189991,472420,80669,56502,...,44708,74449,33829,152986,50995,23067,59691,27844,110602,36867
2,강남,30,487476,254316,190387,932179,310726,516703,266541,215168,...,171525,303086,146677,621288,207096,66658,174059,106065,346782,115594
3,강남구청,202,135107,40956,27120,203183,67728,149142,45157,30107,...,44780,48108,19545,112433,37478,22120,27641,14221,63982,21327
4,강동,142,67354,47432,101500,216286,72095,71116,50529,108876,...,25707,51545,56118,133370,44457,16092,35817,45066,96975,32325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,홍제,63,34656,53147,107203,195006,65002,36892,54266,114807,...,22788,50973,58130,131891,43964,14225,34595,43636,92456,30819
244,화곡,117,41853,70904,200771,313528,104509,43569,72929,214785,...,24877,71924,102234,199035,66345,14563,48860,76953,140376,46792
245,화랑대(서울여대입구),183,14162,20286,68335,102783,34261,15395,21430,73136,...,10344,20516,28914,59774,19925,7090,15444,21830,44364,14788
246,회현(남대문시장),102,166552,90956,19962,277470,92490,180303,95701,22068,...,64911,100141,16167,181219,60406,32740,43416,12094,88250,29417


In [41]:
qt1_data = pd.merge(group_test,testing3, how='outer',on=['link', '역명'])
qt1_data.dropna(axis=0, inplace=True)
qt1_data.to_csv("./qt1_money_popular_data.csv", encoding='utf-8-sig')
qt1_data

C:\Users\La_Elya\AppData\Local\Temp\ipykernel_19968\4116270757.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  qt1_data = pd.merge(group_test,testing3, how='outer',on=['link', '역명'])


,link,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,자치구,동,서비스_업종_코드,...,토_00시11시,토_11시17시,토_17시24시,토_총합,토_평균,일_00시11시,일_11시17시,일_17시24시,일_총합,일_평균
0,0.0,21571.0,2021.0,1.0,D,발달상권,2120043.0,중구,회현동,CS100005,...,101999,165188,108733,375920,125307,57909,126607,106419,290935,96978
1,1.0,62783.0,2021.0,1.0,D,발달상권,2120021.0,중구,소공동,CS100005,...,55397,84498,26881,166776,55592,35469,57413,20082,112964,37655
2,2.0,20452.0,2021.0,1.0,D,발달상권,2120006.0,종로구,종로1.2.3.4가동,CS100005,...,47018,90863,30531,168412,56137,27769,61441,20049,109259,36420
3,3.0,20565.0,2021.0,1.0,D,발달상권,2120009.0,종로구,종로1.2.3.4가동,CS100005,...,85797,243962,73228,402987,134329,45097,164100,45170,254367,84789
4,4.0,20721.0,2021.0,1.0,D,발달상권,2120014.0,종로구,종로5.6가동,CS100005,...,50221,109286,24342,183849,61283,16478,56889,15106,88473,29491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,232.0,28576.0,2021.0,1.0,D,발달상권,2120233.0,송파구,방이2동,CS100005,...,12120,19009,13703,44832,14944,7402,14252,10870,32524,10841
170,233.0,65496.0,2021.0,1.0,A,골목상권,2114083.0,송파구,석촌동,CS100005,...,13810,34659,26928,75397,25132,7879,24665,21497,54041,18014
171,234.0,28490.0,2021.0,1.0,D,발달상권,2120230.0,송파구,석촌동,CS100005,...,12068,22433,23957,58458,19486,6273,15358,18831,40462,13487
172,235.0,28641.0,2021.0,1.0,D,발달상권,2120235.0,송파구,문정2동,CS100005,...,24746,39209,28560,92515,30838,14467,24987,21841,61295,20432
